In [26]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [27]:
""""
- For the Transaction Path table:
    - Make sure field naming convention matches the other tables
      i.e. instead of Account_From it should be Account From
- Filter out the cancelled transactions
- Split the flow into incoming and outgoing transactions 
- Bring the data together with the Balance as of 31st Jan 
- Work out the order that transactions occur for each account
Hint: where multiple transactions happen on the same day, assume the highest value transactions happen first
- Use a running sum to calculate the Balance for each account on each day (hint)
- The Transaction Value should be null for 31st Jan, as this is the starting balance
"""

'"\n- For the Transaction Path table:\n    - Make sure field naming convention matches the other tables\n      i.e. instead of Account_From it should be Account From\n- Filter out the cancelled transactions\n- Split the flow into incoming and outgoing transactions \n- Bring the data together with the Balance as of 31st Jan \n- Work out the order that transactions occur for each account\nHint: where multiple transactions happen on the same day, assume the highest value transactions happen first\n- Use a running sum to calculate the Balance for each account on each day (hint)\n- The Transaction Value should be null for 31st Jan, as this is the starting balance\n'

In [28]:
#Import Data
account = pd.read_csv("Account Information.csv")
detail = pd.read_csv("Transaction Detail.csv")
path = pd.read_csv("Transaction Path.csv")

In [29]:
#Rename columns in Transaction Path csv
path.rename({'Account_To': 'Account To', 'Account_From': 'Account From'}, axis =1, inplace= True)

In [30]:
#Filter to only non-cancelled transactions
detail = detail[detail['Cancelled?'] == 'N']

In [31]:
#Combine Transaction details with transaction path data
transactions = pd.merge(detail, right =path,on= 'Transaction ID', how='left')

In [32]:
#Split transactions to incoming and outgoing
incoming = transactions.drop('Account From', axis=1)
outgoing = transactions.drop('Account To', axis=1)
outgoing['Value'] = outgoing['Value']*-1

In [33]:
#Filter accounts data to only relevant columns
account = account[['Account Number', 'Balance Date', 'Balance']]

In [34]:
#Change date columns to type datetime64 for consistency

account['Balance Date'] = account['Balance Date'].astype('datetime64[D]')
outgoing['Transaction Date'] = outgoing['Transaction Date'].astype('datetime64[D]')
incoming['Transaction Date'] = incoming['Transaction Date'].astype('datetime64[D]')

In [35]:
#Merge incoming transactions with the accounts
incoming_merg = pd.merge(account, right = incoming, left_on= 'Account Number', right_on='Account To', how = 'inner')
incoming_merg['Account To'] = incoming_merg['Account To'].astype('int64')

#Merge outgoing transactions with the accounts
outgoing_merge = pd.merge(account, right= outgoing, left_on='Account Number', right_on='Account From', how = 'inner')
outgoing_merge['Account From'] = outgoing_merge['Account From'].astype('int64')

In [36]:
#Combine everything into one dataframe
df = pd.concat([incoming_merg,outgoing_merge])
df.drop(['Cancelled?', 'Account To', 'Account From'], axis=1 , inplace= True)

In [37]:
"""Not the ideal solution but genuinely couldn't think of another way to do it. If anyone has a better way of doing this please reach out. 
1- Get a unique list of Account Numbers to loops through
2- Add a row with the starting balance on starting date 31st of Jan 2023
"""

unique_list = df['Account Number'].unique().tolist()


for id in unique_list:
    df2 = {'Account Number': id, 
           'Balance Date': pd.to_datetime('2023-01-31'), 
           'Balance': account[account['Account Number'] == id]['Balance'].iloc[-1],
           'Transaction ID': 'null',
           'Transaction Date': pd.to_datetime('2023-01-31'),
           'Value': account[account['Account Number'] == id]['Balance'].iloc[-1]}
    df = df.append(df2, ignore_index = True)
  

In [38]:
#Sort table and show it. 
df.sort_values(by=['Account Number','Transaction Date','Value'], inplace=True, ascending=[True,True,False])
df

,Account Number,Balance Date,Balance,Transaction ID,Transaction Date,Value
17378,10005367,2023-01-31,728.25,null,2023-01-31,728.25
8689,10005367,2023-01-31,728.25,1753186544,2023-02-02,-1097.60
0,10005367,2023-01-31,728.25,601263790,2023-02-04,144.75
8690,10005367,2023-01-31,728.25,8072094629,2023-02-04,-67.01
8691,10005367,2023-01-31,728.25,3011215596,2023-02-05,-108.26
...,...,...,...,...,...,...
8687,99937043,2023-01-31,918.42,2441743678,2023-02-05,52.36
17375,99937043,2023-01-31,918.42,2016922982,2023-02-06,-92.42
17376,99937043,2023-01-31,918.42,7354862347,2023-02-09,-122.91
17377,99937043,2023-01-31,918.42,3771418488,2023-02-11,-125.33


In [39]:
#Calculate running sum
df['Balance New'] = df.groupby('Account Number')['Value'].cumsum()

In [40]:
#Clean the resulting dataframe for the final output
df =df[['Account Number', 'Transaction Date', 'Value', 'Balance New']] #Keep only needed columns
df.rename({'Transaction Date': 'Balance Date', 'Value': 'Transaction Value', 'Balance New': 'Balance'}, axis =1, inplace= True) #rename columns
df.loc[df['Balance Date'] == '2023-01-31', 'Transaction Value'] = None #Set Transaction Value to Null at the starting date


C:\Users\Dell\AppData\Local\Temp\ipykernel_16200\3513722624.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename({'Transaction Date': 'Balance Date', 'Value': 'Transaction Value', 'Balance New': 'Balance'}, axis =1, inplace= True) #rename columns


In [44]:
#Output
df.to_csv('output.csv', index= False)
df

#Note: The result is 20,365 rows while the solution shows it should be 20,378 rows. No idea where the missing rows are.
#If anyone can figure it out please feel free to reach out to me. 

,Account Number,Balance Date,Transaction Value,Balance
17378,10005367,2023-01-31,NaN,728.25
8689,10005367,2023-02-02,-1097.60,-369.35
0,10005367,2023-02-04,144.75,-224.60
8690,10005367,2023-02-04,-67.01,-291.61
8691,10005367,2023-02-05,-108.26,-399.87
...,...,...,...,...
8687,99937043,2023-02-05,52.36,970.18
17375,99937043,2023-02-06,-92.42,877.76
17376,99937043,2023-02-09,-122.91,754.85
17377,99937043,2023-02-11,-125.33,629.52
